In [41]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
import os
import time
import pandas as pd

In [43]:
service = Service("C://Users//servi//inter-scraping//chromedriver-win64//chromedriver-win64//chromedriver.exe")
chrome_profile_path = r'C:\Users\servi\AppData\Local\Google\Chrome\User Data\Default'

# Set up Chrome options
# Configuração das opções do Chrome
options = webdriver.ChromeOptions()
options.add_argument(f"user-data-dir={chrome_profile_path}")
options.add_experimental_option("detach", True)  # Isso permite que o Chrome permaneça aberto após o script terminar

# Inicializa o Chrome com as opções
driver = webdriver.Chrome(service=service)
"""
https://gestao.granitopagamentos.com.br/Login/Index
sandro.leao@creditoessencial.com.br
11S@ndr01003
"""

'\nhttps://gestao.granitopagamentos.com.br/Login/Index\nsandro.leao@creditoessencial.com.br\n11S@ndr01003\n'

In [45]:
def change_window(window_name):
    link_pesquisa = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.LINK_TEXT, window_name))
    )
    try:
        link_pesquisa.click()
    except ElementClickInterceptedException:
        time.sleep(0.5)
    wait_for_processing_to_disappear(driver)

def wait_for_processing_to_disappear(driver, timeout=10):
    """ Aguarda a aba 'Processando...' desaparecer antes de continuar """
    try:
        WebDriverWait(driver, timeout).until(
            EC.invisibility_of_element_located((By.XPATH, "//*[contains(text(), 'Processando')]"))
        )
        print("Aba 'Processando...' desapareceu. Continuando...")
    except Exception as e:
        print(f"Erro ou tempo esgotado ao esperar a aba 'Processando...': {e}")

def get_element_text_by_xpath(xpath, option = 'value'):
    element_input = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, xpath))
            )

    # Capturar o valor do input
    element_text = element_input.get_attribute(option)
    return element_text

def loop_in_lines():
    equipment_data = {
    'name': [], 'cpf_cnpj': [],
    'equipment_id': [], 'pdv': [], 'phone': [], 'email': [],
    'fantasy_name': [], 'branch_of_activity': [], 'cep': [],
    'street_name': [], 'address_number': [], 'address_complement': [], 'neighborhood_name': [],
    'city_name': [], 'state_name': []
    }
    page = 1
    while True:
        client_rows = driver.find_elements(By.XPATH, "//table[@id='tabelaClientes']/tbody/tr")
        for i, client_row in enumerate(client_rows):
            try:
                cpf_cnpj = client_row.find_element(By.XPATH, ".//td[3]").text
                name = client_row.find_element(By.XPATH, ".//td[4]").text
                print(f'{i+1}/{len(client_rows)} da pag. {page}')
                client_row.click()
                time.sleep(0.5)

                phone = get_element_text_by_xpath('//*[@id="tabelaTelefones"]/tbody/tr/td[2]/input')
                email = get_element_text_by_xpath('//*[@id="tabelaEmails"]/tbody/tr/td[2]/input')
                fantasy_name = get_element_text_by_xpath('//*[@id="Fantasia"]')
                branch_of_activity = get_element_text_by_xpath('//*[@id="RamoAtividadePF"]/option', 'text')
                cep = get_element_text_by_xpath('//*[@id="CEP"]')
                street_name = get_element_text_by_xpath('//*[@id="Endereco"]')
                address_number = get_element_text_by_xpath('//*[@id="Numero"]')
                address_complement = get_element_text_by_xpath('//*[@id="Complemento"]')
                neighborhood_name = get_element_text_by_xpath('//*[@id="Bairro"]')
                city_name = get_element_text_by_xpath('//*[@id="Cidade"]/option', 'text')
                state_name = get_element_text_by_xpath('//*[@id="UF"]/option', 'text')
                
                change_window('Equipamentos')
    
                # Espera até que os elementos estejam disponíveis
                max_retries = 5
                retries = 0
                while retries < max_retries:
                    try:
                        equipment_rows = driver.find_elements(By.XPATH, "//table[@id='tabelaPDV']/tbody/tr")
                        
                        if not equipment_rows:
                            break  # Nenhum equipamento encontrado, segue para o próximo cliente
    
                        for equipment_row in equipment_rows:
                            equipment_id = equipment_row.find_element(By.XPATH, ".//td[2]").text
                            pdv = equipment_row.find_element(By.XPATH, ".//td[1]").text
                            equipment_data['name'].append(name)
                            equipment_data['cpf_cnpj'].append(cpf_cnpj)
                            equipment_data['equipment_id'].append(equipment_id)
                            equipment_data['pdv'].append(pdv)
                            equipment_data['phone'].append(phone)
                            equipment_data['email'].append(email)
                            equipment_data['fantasy_name'].append(fantasy_name)
                            equipment_data['cep'].append(cep)
                            equipment_data['street_name'].append(street_name)
                            equipment_data['address_number'].append(address_number)
                            equipment_data['address_complement'].append(address_complement)
                            equipment_data['branch_of_activity'].append(branch_of_activity)
                            equipment_data['neighborhood_name'].append(neighborhood_name)
                            equipment_data['city_name'].append(city_name)
                            equipment_data['state_name'].append(state_name)
    
                        print(f'current_data: {equipment_data}')
    
                        break  # Sai do `while` se tudo der certo
    
                    except StaleElementReferenceException:
                        retries += 1
                        time.sleep(1)  # Tenta novamente após um pequeno delay
    
                change_window('Pesquisa')
    
            except Exception as e:
                change_window('Pesquisa')
    
        # Verifica se o botão "Próximo" está desativado
        try:
            next_button_li = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "tabelaClientes_next"))
            )
            
            if 'disabled' in next_button_li.get_attribute('class'):
                print("Fim da paginação. Encerrando.")
                return equipment_data
                
            next_button = next_button_li.find_element(By.TAG_NAME, "a")
            page += 1
            next_button.click()
            time.sleep(1)
    
        except Exception as e:
            print(f"Erro ao clicar em 'Próximo': {e}")
            return equipment_data  # Sai do loop se houver erro na paginação

def search_by_cpf_cnpj(cpf_cnpj):
    cpf_cnpj_input = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.ID, 'CPF_CNPJPesquisa'))
    )
    cpf_cnpj_input.clear()
    cpf_cnpj_input.send_keys(cpf_cnpj)

    search_button = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.ID, 'btnPesquisarCliente'))
    )
    search_button.click()


In [66]:
equipment_data_general = loop_in_lines()
df = pd.DataFrame(equipment_data_general)
df.to_csv('equipment_data_general.csv')

1/10 da pag. 1


KeyboardInterrupt: 

In [53]:
get_element_text_by_xpath('//*[@id="TipoConta"]/option', 'text')

'Conta Corrente'

In [55]:
df = pd.read_csv('new_equipment_data.csv')

In [57]:
df

,Unnamed: 0.1,Unnamed: 0,name,cpf_cnpj,equipment_id,pdv,phone,email,fantasy_name,branch_of_activity,...,address_complement,neighborhood_name,city_name,state_name,status,bank_name,agency,account_number,account_dv,account_type
0,0,0.0,Filipe Rodrigues de Oliveira,093.727.146-23,JC9900471961,1,(31) 98628-2290,mailtofiliperodrigues18@hotmail.com,STUDIO Filipe Rodrigues,Barbeiro e cabeleireiro(a),...,NaN,União,Belo Horizonte,MG,Configurado,341 - Banco Itaú Unibanco S.A.,3101,42848,4,Conta Corrente
1,1,1.0,Victor Emannuel Medeiros Silva,017.841.496-40,JA8300482382,1,(31) 97360-9966,victoremannuel17@yahoo.com,Victor Emannuel Medeiros Silva,Venda porta-a-porta e venda direta,...,NaN,Ernesto do Nascimento(Barreiro),Belo Horizonte,MG,Configurado,77 - Banco Inter S.A.,1,42584997,0,Conta Corrente
2,2,2.0,Victor Emannuel Medeiros Silva,017.841.496-40,JA8400507804,2,(31) 97360-9966,victoremannuel17@yahoo.com,Victor Emannuel Medeiros Silva,Venda porta-a-porta e venda direta,...,NaN,Ernesto do Nascimento(Barreiro),Belo Horizonte,MG,Configurado,77 - Banco Inter S.A.,1,42584997,0,Conta Corrente
3,3,3.0,Euires Teixeira de Melo,977.615.086-15,NaN,1,(31) 98364-0168,teuires13@gmail.com,Euires Teixeira de Melo,Venda porta-a-porta e venda direta,...,NaN,Jardim Felicidade,Belo Horizonte,MG,Configurado,748 - Banco Cooperativo Sicredi S.A.,221,57665,4,Conta Corrente
4,4,4.0,Francisco José da Silva,029.913.726-05,JC9900476588,1,(31) 98364-0168,teuires13@gmail.com,Euires Teixeira de Melo,Venda porta-a-porta e venda direta,...,NaN,Jardim Felicidade,Belo Horizonte,MG,Configurado,104 - Caixa Econômica Federal,2427,774720641,1,Conta Corrente
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,92,NaN,Luiz Cesar Ribeiro Marques,58.042.976/0001-07,JA7B00250269,1,(21) 96476-6691,luizvalentina220819@gmail.com,Luiz Cesar Ribeiro Marques,Barbeiro e cabeleireiro(a),...,NaN,Ricardo de Albuquerque,Rio de Janeiro,RJ,REPROVADO,756 - Banco Cooperativo Sicoob S.A.,4338,13943,2,Conta Corrente
155,93,NaN,Jose Milton da Conceição,408.899.046-34,JC9900467655,1,(31) 98954-7596,joseconceicao024@gmail.com,Jose Milton da Conceição,Venda porta-a-porta e venda direta,...,C,Conjunto Jardim Filadélfia,Belo Horizonte,MG,Configurado,1 - Banco do Brasil S.A.,2556,179924056,5,Conta Corrente
156,94,NaN,Jose Milton da Conceição,408.899.046-34,NaN,2,(31) 98954-7596,joseconceicao024@gmail.com,Jose Milton da Conceição,Venda porta-a-porta e venda direta,...,C,Conjunto Jardim Filadélfia,Belo Horizonte,MG,Configurado,1 - Banco do Brasil S.A.,2556,179924056,5,Conta Corrente
157,95,NaN,Luis Fillipe do Rosário Lima,57.282.802/0001-59,JA8B00856612,1,(27) 99619-9620,luisfillipe2023@gmail.com,Luis Fillipe do Rosário Lima,Agência de turismo e guia turístico,...,torre 1 APTO 801,Nossa Senhora da Penha,Vila Velha,ES,Configurado,77 - Banco Inter S.A.,1,38908082,9,Conta Corrente


In [76]:
cpf_cnpj_equipment_data = {}
for cpf_cnpj in cpf_cnpj_data:
    if cpf_cnpj not in equipment_data_general['cpf_cnpj']:
        search_by_cpf_cnpj(cpf_cnpj)
        user_data = loop_in_lines()  

        for key in cpf_cnpj_equipment_data:
            cpf_cnpj_transaction_data[key].extend(user_data.get(key, []))


1/10 da pag. 1
Aba 'Processando...' desapareceu. Continuando...
Aba 'Processando...' desapareceu. Continuando...
Aba 'Processando...' desapareceu. Continuando...
Aba 'Processando...' desapareceu. Continuando...
Aba 'Processando...' desapareceu. Continuando...
Aba 'Processando...' desapareceu. Continuando...
Aba 'Processando...' desapareceu. Continuando...
Aba 'Processando...' desapareceu. Continuando...
Aba 'Processando...' desapareceu. Continuando...
Aba 'Processando...' desapareceu. Continuando...
Fim da paginação. Encerrando.
1/1 da pag. 1
Aba 'Processando...' desapareceu. Continuando...
Fim da paginação. Encerrando.
1/1 da pag. 1
Aba 'Processando...' desapareceu. Continuando...
Fim da paginação. Encerrando.
1/1 da pag. 1
Aba 'Processando...' desapareceu. Continuando...
Fim da paginação. Encerrando.
1/1 da pag. 1
Aba 'Processando...' desapareceu. Continuando...
Fim da paginação. Encerrando.
1/1 da pag. 1
Aba 'Processando...' desapareceu. Continuando...
Fim da paginação. Encerrando.


KeyboardInterrupt: 

In [82]:
pd.read_csv('equipment_data_general.csv')

,Unnamed: 0,name,cpf_cnpj,equipment_id,pdv,phone,email,fantasy_name,branch_of_activity,cep,street_name,address_number,address_complement,neighborhood_name,city_name,state_name,status
0,0,Victor Emannuel Medeiros Silva,017.841.496-40,JA8300482382,1,(31) 97360-9966,victoremannuel17@yahoo.com,Victor Emannuel Medeiros Silva,Venda porta-a-porta e venda direta,30668-450,Rua Flávio de Paiva Pereira,143,NaN,Ernesto do Nascimento(Barreiro),Belo Horizonte,MG,Configurado
1,1,Victor Emannuel Medeiros Silva,017.841.496-40,JA8400507804,2,(31) 97360-9966,victoremannuel17@yahoo.com,Victor Emannuel Medeiros Silva,Venda porta-a-porta e venda direta,30668-450,Rua Flávio de Paiva Pereira,143,NaN,Ernesto do Nascimento(Barreiro),Belo Horizonte,MG,Configurado
2,2,Euires Teixeira de Melo,977.615.086-15,NaN,1,(31) 98364-0168,teuires13@gmail.com,Euires Teixeira de Melo,Venda porta-a-porta e venda direta,31742-468,Beco Flor das Acácias,38,NaN,Jardim Felicidade,Belo Horizonte,MG,Configurado
3,3,Francisco José da Silva,029.913.726-05,JC9900476588,1,(31) 98563-5448,franciscojose11011974@gmail.com,Francisco José da Silva,Venda porta-a-porta e venda direta,31360-540,Rua Alex George do Nascimento,200,NaN,Confisco,Belo Horizonte,MG,Configurado
4,4,54.112.002 AYARA ANTONIETA BAZILIO BARBOSA,54.112.002/0001-00,JA6600033830,1,(31) 99394-9624,ayarabazilio52@gmail.com,Ayara Antonieta Bazilio Barbosa,Venda porta-a-porta e venda direta,30881-040,Rua Taciba,80,NaN,Coqueiros,Belo Horizonte,MG,Configurado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,142,Leandro Ferreira Do Carmo,50.350.607/0001-70,NaN,1,(22) 99899-3522,leandromkf15@gmail.com,LFC INOVA SIMPLES,Professor particular,28300-000,Rua Travessa Tiago Rodrigues Rocha,35,Altos,Cidade Nova,Itaperuna,RJ,Configurado
143,143,Gleidson Widiney Divino da Silva,36.644.916/0001-21,NaN,1,(33) 98838-3704,glaydson98@gmail.com,Lanchonete da Sol,Professor particular,35105-000,Coronel Roberto Soares Ferreira,331,Casa,Baguari,Governador Valadares,MG,Configurado
144,144,Luciene dos Santos Leite Alves,024.766.074-44,NaN,1,(21) 99955-6827,lucienesantosleitealves@gmail.com,Luciene dos Santos Leite Alves,"Venda de itens variados / 1,99",21921-400,Rua Ebro,14,Loja A,Jardim Carioca,Rio de Janeiro,RJ,Configurado
145,145,Romário Raffa costa cruz,29.569.793/0001-46,NaN,1,(99) 98553-4028,romarioraffa120@gmail.com,Bar fundo de quintal,"Venda de itens variados / 1,99",65916-155,Rua Rio Grande do Norte,2219,Casa,Bacuri,Imperatriz,MA,Configurado


In [86]:
df2 = pd.read_excel('teste_completo.xlsx')

In [98]:
df2 = df2.dropna(subset=['city_name']).iloc[:, 2:]

In [100]:
df1 = pd.read_csv('equipment_data_general.csv')

In [104]:
pd.concat([df1, df2]).to_excel('teste_completo.xlsx')